# MusicGen-Large Finetuning Loop (Sequential Processing) - Transformers Version

このノートブックは、大規模なデータセットをZIPファイル単位で順次処理（解凍→学習→削除）しながらMusicGen-Largeをファインチューニングします。
Hugging Face Transformersライブラリを使用します。

## 前提条件
1. Google Driveに以下のデータがあること
   - `MyData/Archive_wavs/metadata.jsonl`: 全データのメタデータ
   - `MyData/Archive_wavs/archive_batch_xxxx.zip`: 音声データのZIPファイル群
2. A100 GPU推奨（VRAM容量のため）
3. **WandB API Key**: Colabのシークレット（鍵マーク）に `WANDB_API_KEY` という名前で登録してください。

In [9]:
# @title 1. 環境設定とライブラリインストール
import os
import subprocess
import sys

print("Installing libraries...")

# CUDA 12.6対応のPyTorch (Nightly or Pre-release)
# 注意: ユーザー指定によりCUDA 12.6をターゲットにします。
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu126

# Hugging Face Libraries & WandB
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U datasets accelerate bitsandbytes wandb

print("Installation complete.")

Installing libraries...
Looking in indexes: https://download.pytorch.org/whl/nightly/cu126
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-1phx3fi0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-1phx3fi0
  Resolved https://github.com/huggingface/transformers.git to commit 91865a69db92496b312847a41cf82b48c8589826
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 56.6 MB/s  0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.23.0
    Uninstalling wandb-0.23.0:
      Successfully uninstalled wandb-0.23.0
Installation complete.


In [10]:
# @title 1.5 WandB ログイン
import wandb
from google.colab import userdata

try:
    wandb_api_key = userdata.get('WANDB_API_KEY')
    wandb.login(key=wandb_api_key)
    print("Logged in to WandB successfully.")
except Exception as e:
    print(f"WandB login failed: {e}")
    print("Please ensure 'WANDB_API_KEY' is set in Colab secrets.")

WandB login failed: Requesting secret WANDB_API_KEY timed out. Secrets can only be fetched when running from the Colab UI.
Please ensure 'WANDB_API_KEY' is set in Colab secrets.


In [11]:
# @title 2. Google Drive マウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# @title 3. パスと設定の定義
import os
from pathlib import Path

# --- ユーザー設定エリア ---
DRIVE_ROOT = Path('/content/drive/MyDrive')
DATA_ROOT = DRIVE_ROOT / 'MyData/Archive_Wavs'
METADATA_PATH = DATA_ROOT / 'metadata.jsonl'
ZIP_DIR = DATA_ROOT

# 出力先（チェックポイント保存場所）
OUTPUT_DIR = DRIVE_ROOT / 'MusicGen_Finetuning_Output'
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# 一時作業ディレクトリ（Colabローカル）
TEMP_WORK_DIR = Path('/content/temp_work')
TEMP_DATA_DIR = TEMP_WORK_DIR / 'data'

TEMP_DATA_DIR.mkdir(exist_ok=True, parents=True)

print(f"Metadata: {METADATA_PATH}")
print(f"Zip Dir: {ZIP_DIR}")
print(f"Output Dir: {OUTPUT_DIR}")

Metadata: /content/drive/MyDrive/MyData/Archive_wavs/metadata.jsonl
Zip Dir: /content/drive/MyDrive/MyData/Archive_wavs
Output Dir: /content/drive/MyDrive/MusicGen_Finetuning_Output


In [13]:
# @title 4. ヘルパー関数の定義
import json
import shutil
import subprocess
import glob
import torchaudio
from datasets import load_dataset, Audio

def extract_zip(zip_path, extract_to):
    """ZIPファイルを指定ディレクトリに解凍する"""
    print(f"Extracting {zip_path} to {extract_to}...")
    if extract_to.exists():
        shutil.rmtree(extract_to)
    extract_to.mkdir(parents=True, exist_ok=True)

    subprocess.run(['unzip', '-q', str(zip_path), '-d', str(extract_to)], check=True)
    print("Extraction complete.")

def create_batch_metadata(main_metadata_path, current_wav_dir, output_jsonl_path):
    """
    メインのmetadata.jsonlから、現在解凍されているファイルに対応するエントリのみを抽出し、
    パスをColab上の絶対パスに書き換えて新しいjsonlを作成する。
    """
    print(f"Creating batch metadata at {output_jsonl_path}...")

    extracted_files = list(current_wav_dir.rglob('*.wav'))
    extracted_files_map = {f.name: f for f in extracted_files}

    valid_entries = []

    with open(main_metadata_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                entry = json.loads(line)
                orig_path = entry.get('path', '')
                filename = os.path.basename(orig_path)

                if filename in extracted_files_map:
                    # パスを絶対パスに更新
                    entry['path'] = str(extracted_files_map[filename])
                    # TransformersのDatasetで読み込むために 'audio' キーにパスを入れるのが一般的だが
                    # ここでは後処理でロードするため 'path' のままでもOK。
                    # ただし、datasets libraryのAudio機能を使うなら 'audio': path が便利。
                    entry['audio'] = str(extracted_files_map[filename])
                    valid_entries.append(entry)
            except json.JSONDecodeError:
                continue

    if not valid_entries:
        print("Warning: No matching metadata found for extracted files.")
        return False

    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for entry in valid_entries:
            f.write(json.dumps(entry) + '\n')

    print(f"Created metadata with {len(valid_entries)} entries.")
    return True

def preprocess_function(examples, processor, audio_column_name="audio", text_column_name="caption"):
    """データセットの前処理関数"""
    audio_arrays = [x["array"] for x in examples[audio_column_name]]
    sampling_rate = examples[audio_column_name][0]["sampling_rate"]

    # テキストの処理
    # metadataのキーが 'caption' か 'text' か 'description' か確認が必要。
    # ここでは 'caption' または 'text' を探す。
    texts = []
    for i in range(len(audio_arrays)):
        # 柔軟にキーを探す
        text = examples.get(text_column_name, [""] * len(audio_arrays))[i]
        if not text and "text" in examples:
            text = examples["text"][i]
        if not text and "description" in examples:
            text = examples["description"][i]
        texts.append(text if text else "")

    inputs = processor(
        audio=audio_arrays,
        sampling_rate=sampling_rate,
        text=texts,
        padding=True,
        truncation=True,
        max_length=256, # テキストの最大長
        return_tensors="pt",
    )
    return inputs

In [14]:
# @title 5. メインループ実行
import torch
from transformers import AutoProcessor, MusicgenForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Audio

# モデルとプロセッサの準備
MODEL_ID = "facebook/musicgen-large"
print(f"Loading model: {MODEL_ID}...")
processor = AutoProcessor.from_pretrained(MODEL_ID)
model = MusicgenForConditionalGeneration.from_pretrained(MODEL_ID)
model.train()

# ZIPファイルリスト取得
zip_files = sorted(list(ZIP_DIR.glob('archive_batch_*.zip')))
print(f"Found {len(zip_files)} zip files.")

# 以前のチェックポイントがあればロード（簡易実装）
latest_checkpoint_path = OUTPUT_DIR / 'latest_checkpoint'
if latest_checkpoint_path.exists():
    print(f"Resuming from {latest_checkpoint_path}...")
    model = MusicgenForConditionalGeneration.from_pretrained(latest_checkpoint_path)

# GPU設定
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for i, zip_file in enumerate(zip_files):
    print(f"\n{'='*40}")
    print(f"Processing Batch {i+1}/{len(zip_files)}: {zip_file.name}")
    print(f"{'='*40}")

    # 1. 解凍
    extract_zip(zip_file, TEMP_DATA_DIR)

    # 2. メタデータ作成
    batch_metadata_path = TEMP_WORK_DIR / 'batch.jsonl'
    success = create_batch_metadata(METADATA_PATH, TEMP_DATA_DIR, batch_metadata_path)

    if not success:
        print("Skipping this batch due to metadata error.")
        continue

    # 3. データセット準備
    dataset = load_dataset("json", data_files=str(batch_metadata_path), split="train")
    dataset = dataset.cast_column("audio", Audio(sampling_rate=processor.feature_extractor.sampling_rate))

    # 前処理の適用
    print("Preprocessing dataset...")
    encoded_dataset = dataset.map(
        lambda x: preprocess_function(x, processor),
        batched=True,
        remove_columns=dataset.column_names,
        batch_size=4 # メモリに応じて調整
    )

    # 4. トレーニング設定
    # バッチごとにTrainerを作り直すが、modelは同じオブジェクトを使い回すことで学習を継続する
    training_args = TrainingArguments(
        output_dir=str(TEMP_WORK_DIR / "results"),
        per_device_train_batch_size=2, # A100ならもう少し増やせるかも
        gradient_accumulation_steps=4,
        learning_rate=1e-5,
        num_train_epochs=5, # 1バッチあたりのエポック数
        save_steps=1000, # バッチ内での保存頻度（必要なら）
        logging_steps=10,
        fp16=True, # A100/V100ならTrue推奨
        save_total_limit=1,
        remove_unused_columns=False,
        dataloader_num_workers=2,
        report_to="wandb", # WandB有効化
        run_name=f"musicgen-finetuning-batch-{i+1}", # バッチごとにRun名を分ける
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_dataset,
    )

    print("Starting training for this batch...")
    trainer.train()

    # 5. モデル保存
    # バッチ完了ごとにDriveへ保存
    save_path = OUTPUT_DIR / f'checkpoint_batch_{i+1}'
    print(f"Saving model to {save_path}...")
    model.save_pretrained(save_path)
    processor.save_pretrained(save_path)

    # 最新版として上書き
    latest_path = OUTPUT_DIR / 'latest_checkpoint'
    model.save_pretrained(latest_path)
    processor.save_pretrained(latest_path)

    # 6. クリーンアップ
    print("Cleaning up temp data...")
    shutil.rmtree(TEMP_DATA_DIR)
    TEMP_DATA_DIR.mkdir(exist_ok=True)
    # Trainerのクリーンアップ（メモリ解放のため）
    del trainer
    del dataset
    del encoded_dataset
    torch.cuda.empty_cache()

print("All batches processed.")

Loading model: facebook/musicgen-large...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/995 [00:00<?, ?it/s]

MusicgenForConditionalGeneration LOAD REPORT from: facebook/musicgen-large
Key                                           | Status     |  | 
----------------------------------------------+------------+--+-
decoder.model.decoder.embed_positions.weights | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Found 0 zip files.
All batches processed.
